# Hybrid Healing Workflows

## Overview
This notebook implements hybrid healing workflows that combine rule-based and AI-driven remediation approaches. It intelligently routes decisions between deterministic rules and ML models, optimizing for both reliability and adaptability.

## Prerequisites
- Completed: `ai-driven-decision-making.ipynb`
- Rule-based remediation logic available
- AI decision making models deployed
- Coordination engine accessible

## Learning Objectives
- Combine rule-based and AI-driven approaches
- Route decisions intelligently
- Optimize remediation success rates
- Handle edge cases and failures
- Track hybrid workflow performance

## Key Concepts
- **Hybrid Approach**: Combine deterministic and ML-based decisions
- **Decision Routing**: Route to best approach for each scenario
- **Fallback Chains**: Multiple fallback strategies
- **Success Optimization**: Maximize healing success
- **Performance Tracking**: Monitor hybrid workflow metrics

## Setup Section

In [ ]:
import sys
import os
import json
import logging
from pathlib import Path
from datetime import datetime, timedelta
import pandas as pd
import numpy as np

# Setup path for utils module - works from any directory
def find_utils_path():
    """Find utils path regardless of current working directory"""
    possible_paths = [
        Path(__file__).parent.parent / 'utils' if '__file__' in dir() else None,
        Path.cwd() / 'notebooks' / 'utils',
        Path.cwd().parent / 'utils',
        Path('/workspace/repo/notebooks/utils'),
        Path('/opt/app-root/src/notebooks/utils'),
    ]
    for p in possible_paths:
        if p and p.exists() and (p / 'common_functions.py').exists():
            return str(p)
    return None

utils_path = find_utils_path()
if utils_path:
    sys.path.insert(0, utils_path)
    print(f"✅ Utils path found: {utils_path}")

# Try to import common functions, with fallback
try:
    from common_functions import setup_environment
    print("✅ Common functions imported")
except ImportError as e:
    print(f"⚠️ Using fallback setup_environment")
    def setup_environment():
        os.makedirs('/opt/app-root/src/data/processed', exist_ok=True)
        os.makedirs('/opt/app-root/src/models', exist_ok=True)
        return {'data_dir': '/opt/app-root/src/data', 'models_dir': '/opt/app-root/src/models'}

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Setup environment
env_info = setup_environment()
logger.info(f"Environment ready: {env_info}")

# Define paths
DATA_DIR = Path('/opt/app-root/src/data')
PROCESSED_DIR = DATA_DIR / 'processed'
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

# Configuration
NAMESPACE = 'self-healing-platform'
AI_CONFIDENCE_THRESHOLD = 0.75
RULE_PRIORITY = 0.8  # Weight for rule-based decisions
AI_PRIORITY = 0.2    # Weight for AI decisions

logger.info(f"Hybrid healing workflows initialized")

## Implementation Section

### 1. Define Hybrid Decision Router

In [ ]:
def route_decision(anomaly_type, confidence, severity):
    """
    Route decision to best approach (rule-based or AI-driven).
    
    Args:
        anomaly_type: Type of anomaly detected
        confidence: AI model confidence score
        severity: Anomaly severity level
    
    Returns:
        Routing decision
    """
    try:
        # Known anomaly types - use rule-based
        known_types = ['high_cpu', 'high_memory', 'pod_crash', 'network_issue']
        
        if anomaly_type in known_types and severity in ['high', 'critical']:
            # Use rule-based for known, severe anomalies
            route = 'rule_based'
            reason = 'Known anomaly type with high severity'
        elif confidence >= AI_CONFIDENCE_THRESHOLD and severity != 'critical':
            # Use AI-driven for high confidence, non-critical
            route = 'ai_driven'
            reason = f'High confidence AI decision ({confidence:.2%})'
        elif severity == 'critical':
            # Use rule-based for critical anomalies
            route = 'rule_based'
            reason = 'Critical severity - use deterministic rules'
        else:
            # Use hybrid approach
            route = 'hybrid'
            reason = 'Combine both approaches for optimal decision'
        
        routing_decision = {
            'route': route,
            'reason': reason,
            'anomaly_type': anomaly_type,
            'confidence': confidence,
            'severity': severity,
            'timestamp': datetime.now().isoformat()
        }
        
        logger.info(f"Routing decision: {route} - {reason}")
        return routing_decision
    except Exception as e:
        logger.error(f"Routing error: {e}")
        return {'route': 'rule_based', 'error': str(e)}

# Test routing
test_cases = [
    ('high_cpu', 0.95, 'critical'),
    ('unknown_anomaly', 0.85, 'high'),
    ('pod_crash', 0.65, 'high'),
]

for anomaly_type, confidence, severity in test_cases:
    routing = route_decision(anomaly_type, confidence, severity)
    print(json.dumps(routing, indent=2, default=str))
    print()

### 2. Implement Hybrid Decision Making

In [ ]:
def make_hybrid_decision(anomaly_data, rule_decision, ai_decision):
    """
    Make hybrid decision combining rule-based and AI approaches.
    
    Args:
        anomaly_data: Detected anomaly
        rule_decision: Rule-based decision
        ai_decision: AI-driven decision
    
    Returns:
        Hybrid decision
    """
    try:
        # Combine decisions with weighted voting
        rule_score = 1.0 if rule_decision.get('should_execute') else 0.0
        ai_score = ai_decision.get('confidence', 0.5)
        
        # Weighted combination
        hybrid_score = (rule_score * RULE_PRIORITY) + (ai_score * AI_PRIORITY)
        
        # Determine final action
        if hybrid_score >= 0.75:
            final_action = 'execute'
            confidence = hybrid_score
        elif hybrid_score >= 0.5:
            final_action = 'execute_with_monitoring'
            confidence = hybrid_score
        else:
            final_action = 'require_approval'
            confidence = hybrid_score
        
        hybrid_decision = {
            'final_action': final_action,
            'hybrid_score': hybrid_score,
            'confidence': confidence,
            'rule_contribution': rule_score * RULE_PRIORITY,
            'ai_contribution': ai_score * AI_PRIORITY,
            'timestamp': datetime.now().isoformat()
        }
        
        logger.info(f"Hybrid decision: {final_action} (score: {hybrid_score:.2f})")
        return hybrid_decision
    except Exception as e:
        logger.error(f"Hybrid decision error: {e}")
        return {'error': str(e)}

# Test hybrid decision
rule_dec = {'should_execute': True, 'action': 'restart'}
ai_dec = {'confidence': 0.88, 'action_level': 'moderate'}
hybrid_dec = make_hybrid_decision({'metric': 85}, rule_dec, ai_dec)
print(json.dumps(hybrid_dec, indent=2, default=str))

### 3. Implement Fallback Chains

In [ ]:
def execute_with_fallback(hybrid_decision, namespace):
    """
    Execute remediation with fallback chain.
    
    Args:
        hybrid_decision: Hybrid decision
        namespace: Kubernetes namespace
    
    Returns:
        Execution result with fallback tracking
    """
    try:
        final_action = hybrid_decision.get('final_action', 'require_approval')
        
        # Fallback chain
        fallback_chain = [
            {'action': 'execute', 'timeout': 5},
            {'action': 'execute_with_monitoring', 'timeout': 10},
            {'action': 'require_approval', 'timeout': 30}
        ]
        
        execution_result = {
            'executed': final_action != 'require_approval',
            'action': final_action,
            'fallback_chain': fallback_chain,
            'timestamp': datetime.now().isoformat()
        }
        
        if final_action == 'execute':
            logger.info(f"Executing remediation immediately")
            execution_result['status'] = 'executed'
        elif final_action == 'execute_with_monitoring':
            logger.info(f"Executing with monitoring")
            execution_result['status'] = 'executing_monitored'
        else:
            logger.info(f"Awaiting approval")
            execution_result['status'] = 'pending_approval'
        
        return execution_result
    except Exception as e:
        logger.error(f"Execution error: {e}")
        return {'executed': False, 'error': str(e)}

# Execute with fallback
execution = execute_with_fallback(hybrid_dec, NAMESPACE)
print(json.dumps(execution, indent=2, default=str))

### 4. Track Hybrid Workflow Performance

In [ ]:
# Create hybrid workflow tracking dataframe
workflow_tracking = pd.DataFrame([
    {
        'timestamp': datetime.now().isoformat(),
        'anomaly_type': np.random.choice(['high_cpu', 'high_memory', 'pod_crash', 'network_issue']),
        'route': np.random.choice(['rule_based', 'ai_driven', 'hybrid']),
        'hybrid_score': np.random.uniform(0.5, 0.99),
        'executed': np.random.choice([True, False]),
        'success': np.random.choice([True, False]),
        'recovery_time_seconds': np.random.randint(5, 60)
    }
    for _ in range(20)  # Simulate 20 workflows
])

# Save tracking data
tracking_file = PROCESSED_DIR / 'hybrid_workflow_tracking.parquet'
workflow_tracking.to_parquet(tracking_file)

logger.info(f"Saved hybrid workflow tracking data")
print(workflow_tracking.to_string())

## Validation Section

In [ ]:
# Verify outputs
assert 'final_action' in hybrid_dec, "No final action in hybrid decision"
assert 'hybrid_score' in hybrid_dec, "No hybrid score in decision"
assert tracking_file.exists(), "Workflow tracking file not created"

success_rate = workflow_tracking['success'].sum() / len(workflow_tracking)
execution_rate = workflow_tracking['executed'].sum() / len(workflow_tracking)

logger.info(f"✅ All validations passed")
print(f"\nHybrid Healing Workflows Summary:")
print(f"  Workflows Executed: {workflow_tracking['executed'].sum()}/{len(workflow_tracking)}")
print(f"  Success Rate: {success_rate:.1%}")
print(f"  Execution Rate: {execution_rate:.1%}")
print(f"  Average Recovery Time: {workflow_tracking['recovery_time_seconds'].mean():.1f}s")
print(f"\nRouting Distribution:")
print(workflow_tracking['route'].value_counts())

## Integration Section

This notebook integrates with:
- **Input**: Rule-based and AI-driven decisions
- **Output**: Hybrid remediation decisions and outcomes
- **Monitoring**: Workflow performance and success metrics
- **Next**: Phase 4 (Model Serving)

## Next Steps

1. Monitor hybrid workflow performance
2. Optimize routing logic based on outcomes
3. Proceed to Phase 4: Model Serving
4. Deploy models for production inference
5. Implement complete end-to-end workflows

## References

- ADR-003: Self-Healing Platform Architecture
- ADR-012: Notebook Architecture for End-to-End Workflows
- [Ensemble Methods](https://en.wikipedia.org/wiki/Ensemble_learning)
- [Hybrid Systems](https://en.wikipedia.org/wiki/Hybrid_system)